In [ ]:
%matplotlib inline

In [ ]:
import awkward
import matplotlib.pyplot as plt
import vector
import glob
import numpy as np

In [ ]:
datas = []
for fi in glob.glob("../data/grid_check/QCD/*.parquet"):
    data = awkward.from_parquet(fi)
    datas.append(data)

In [ ]:
a = np.random.randn(3,3)

In [ ]:
np.stack([a, a]).shape

In [ ]:
np.concatenate([a, a]).shape

In [ ]:
reco_jet_p4s = awkward.concatenate([d["reco_jet_p4s"] for d in datas])
reco_jet_p4s = vector.awk(
    awkward.zip(
        {
            "mass": reco_jet_p4s.tau,
            "x": reco_jet_p4s.x,
            "y": reco_jet_p4s.y,
            "z": reco_jet_p4s.z,
        }
    )
)

inner_grid = awkward.concatenate([d["inner_grid"] for d in datas])
outer_grid = awkward.concatenate([d["outer_grid"] for d in datas])

njet = len(reco_jet_p4s)
inner_grid = inner_grid.to_numpy().reshape(njet, 30, 11, 11)
outer_grid = outer_grid.to_numpy().reshape(njet, 30, 21, 21)


In [ ]:
plt.hist(reco_jet_p4s.theta, bins=100);

In [ ]:
msk_low_theta = reco_jet_p4s.theta<0.5
msk_high_theta = reco_jet_p4s.theta>2.5
msk_mid_theta = ~(msk_low_theta | msk_high_theta)

In [ ]:
for ivar in range(30):
    fig, (axs0, axs1) = plt.subplots(2,3, figsize=(3*4,2*4))

    plt.sca(axs0[0])
    plt.imshow(inner_grid[msk_low_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$\theta <  0.5$")
    plt.colorbar()

    plt.sca(axs0[1])
    plt.imshow(inner_grid[msk_mid_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$0.5 <  \theta < 2.5$")
    plt.colorbar()

    plt.sca(axs0[2])
    plt.imshow(inner_grid[msk_high_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$\theta > 2.5$")
    plt.colorbar()


    plt.sca(axs1[0])
    plt.imshow(outer_grid[msk_low_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$\theta <  0.5$")
    plt.colorbar()

    plt.sca(axs1[1])
    plt.imshow(outer_grid[msk_mid_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$0.5 <  \theta < 2.5$")
    plt.colorbar()

    plt.sca(axs1[2])
    plt.imshow(outer_grid[msk_high_theta, ivar, :, :].mean(axis=0))
    plt.title(r"$\theta > 2.5$")
    plt.colorbar()

    plt.tight_layout()
    plt.suptitle("mean ivar={}".format(ivar))
    plt.savefig("mean_grid_ivar{}.pdf".format(ivar))
    

In [ ]:
def to_p4(d):
    ret = vector.awk(
        awkward.zip(
            {
                "mass": d.tau,
                "x": d.x,
                "y": d.y,
                "z": d.z,
            }
        )
    )
    return ret

In [ ]:
def load_data(path):
    datas = []
    for fi in glob.glob(path):
        data = awkward.from_parquet(fi)
        datas.append(data)
        
    gen_tau_p4s = to_p4(awkward.concatenate([d["gen_jet_tau_p4s"] for d in datas]))
    reco_tau_p4s = to_p4(awkward.concatenate([d["tau_p4s"] for d in datas]))
    return gen_tau_p4s, reco_tau_p4s

In [ ]:
p0 = load_data("../data/DeepTau/wconv/*.parquet")
p1 = load_data("../data/DeepTau/withcorrectgrid/*.parquet")

In [ ]:
p0[1].pt - p1[1].pt

In [ ]:
plt.hist(
    awkward.to_numpy(p0[1].pt)[p0[0].pt>0]/awkward.to_numpy(p0[0].pt)[p0[0].pt>0],
    bins=np.linspace(0,2,100),
    histtype="step", lw=2
);

plt.hist(
    awkward.to_numpy(p1[1].pt)[p1[0].pt>0]/awkward.to_numpy(p1[0].pt)[p1[0].pt>0],
    bins=np.linspace(0,2,100),
    histtype="step", lw=2
);
plt.yscale("log")